## Supermarket sales

This notebook presents plots derived from historical sales data collected from three distinct supermarkets. The original dataset is accessible at the following URL:
 https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales

The plots presented here will be used as baseline to construct an interactive dash app.

In [1]:
#Libraries 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np

In [2]:
# Replace 'your_file.xlsx' with the path to your Excel file
csv_file = "supermarket_sales.csv"

# Read the Excel file into a Pandas DataFrame
df = pd.read_csv(csv_file)
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [3]:
#Get type of data
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [4]:
#Fix Date; object to Datetime
df['Date'] = pd.to_datetime(df['Date'])
df = df.round(2)

#Get type of data
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Invoice ID               1000 non-null   object        
 1   Branch                   1000 non-null   object        
 2   City                     1000 non-null   object        
 3   Customer type            1000 non-null   object        
 4   Gender                   1000 non-null   object        
 5   Product line             1000 non-null   object        
 6   Unit price               1000 non-null   float64       
 7   Quantity                 1000 non-null   int64         
 8   Tax 5%                   1000 non-null   float64       
 9   Total                    1000 non-null   float64       
 10  Date                     1000 non-null   datetime64[ns]
 11  Time                     1000 non-null   object        
 12  Payment                  1000 non-n

In [5]:
#Get unique values for relevant columns
for col in df.select_dtypes(include=['object']):
  if col not in ("Invoice ID","Time","Date"):
    print(col, df[col].unique())

Branch ['A' 'C' 'B']
City ['Yangon' 'Naypyitaw' 'Mandalay']
Customer type ['Member' 'Normal']
Gender ['Female' 'Male']
Product line ['Health and beauty' 'Electronic accessories' 'Home and lifestyle'
 'Sports and travel' 'Food and beverages' 'Fashion accessories']
Payment ['Ewallet' 'Cash' 'Credit card']


In [6]:
dfs  = df.groupby(["Date","City"]).sum(numeric_only=True).reset_index()
dfs['Total_day'] = dfs.groupby('Date')['Total'].transform(sum)
dfs = dfs.round(2)
dfs

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\917261888.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dfs['Total_day'] = dfs.groupby('Date')['Total'].transform(sum)


,Date,City,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating,Total_day
0,2019-01-01,Mandalay,180.46,24,73.18,1536.70,1463.52,14.28,73.18,22.0,4745.19
1,2019-01-01,Naypyitaw,192.29,20,39.87,837.17,797.30,19.04,39.87,25.5,4745.19
2,2019-01-01,Yangon,287.20,37,112.92,2371.32,2258.40,23.80,112.92,31.5,4745.19
3,2019-01-02,Mandalay,141.46,31,55.36,1162.79,1107.43,19.04,55.36,24.8,1945.49
4,2019-01-02,Naypyitaw,131.49,10,22.65,475.65,453.00,9.52,22.65,12.3,1945.49
...,...,...,...,...,...,...,...,...,...,...,...
258,2019-03-29,Naypyitaw,111.18,14,46.94,985.70,938.76,9.52,46.94,10.7,4023.25
259,2019-03-29,Yangon,143.80,13,46.55,977.61,931.06,9.52,46.55,13.9,4023.25
260,2019-03-30,Mandalay,263.19,19,82.12,1724.46,1642.34,14.28,82.12,16.1,4487.06
261,2019-03-30,Naypyitaw,249.73,25,68.66,1441.98,1373.32,19.04,68.66,27.6,4487.06


In [7]:
#filtering by month
dfs[dfs["Date"].dt.month == 2]

,Date,City,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating,Total_day
91,2019-02-01,Naypyitaw,191.34,24,72.98,1532.60,1459.62,14.28,72.98,19.5,2444.54
92,2019-02-01,Yangon,161.43,16,43.43,911.94,868.51,14.28,43.43,21.3,2444.54
93,2019-02-02,Mandalay,192.24,31,76.64,1609.29,1532.65,19.04,76.64,31.7,4140.96
94,2019-02-02,Naypyitaw,374.76,23,71.99,1511.71,1439.72,28.56,71.99,43.3,4140.96
95,2019-02-02,Yangon,123.94,29,48.57,1019.96,971.39,19.04,48.57,25.4,4140.96
...,...,...,...,...,...,...,...,...,...,...,...
169,2019-02-27,Naypyitaw,280.03,16,72.82,1529.36,1456.54,14.28,72.82,25.1,5859.44
170,2019-02-27,Yangon,285.30,38,116.28,2441.70,2325.43,23.80,116.28,34.2,5859.44
171,2019-02-28,Mandalay,101.39,11,23.56,494.76,471.20,9.52,23.56,13.3,2097.02
172,2019-02-28,Naypyitaw,179.18,10,43.71,917.95,874.24,9.52,43.71,14.2,2097.02


In [8]:
# Enable renderers to save plots as svg files; othewise the html plots won't be displayed once uploaded on Github
# Disable renderers to have interactive inline plots in SVC but not visible once uploaded on Github
# pio.renderers.default = "svg"


#app = Dash(__name__)
#Simple example of barplot
fig = px.bar(dfs[dfs["Date"].dt.month == 2],
             x="Date",
             y="Total",
             color="City",
             text = "Total_day",
             barmode='group',
             text_auto=".2s"
             )
fig.show()

In [9]:
df['Gender'].value_counts()

Gender
Female    501
Male      499
Name: count, dtype: int64

In [10]:
#Simple example of pie
gender_by_month = df[df["Date"].dt.month == 2]["Gender"].value_counts()
fig = px.pie(gender_by_month,
             values=gender_by_month.values,
             names=gender_by_month.index,
             title='Total by City, February',
             hole=.3)
fig.update_traces(textposition='inside',
                  textinfo='percent+value')
#textinfo: flaglist string. Any combination of "label", "text", "value", "percent" joined with a "+" OR "none".


fig.show()

In [11]:
#Pie of Payment types
payment_by_month = df[df["Date"].dt.month == 2]["Payment"].value_counts()
fig = px.pie(payment_by_month,
             values=payment_by_month.values,
             names=payment_by_month.index,
             title='Total by City, February',
             hole=.3)
fig.update_traces(textposition='inside',
                  textinfo='percent+value')
#textinfo: flaglist string. Any combination of "label", "text", "value", "percent" joined with a "+" OR "none".


fig.show()

In [12]:
#Bar plot
customer_type_by_month = df[df["Date"].dt.month == 2]["Customer type"].value_counts()
df_ct = customer_type_by_month.reset_index()
df_ct.columns = ['Customer Type', 'Count']
#Adding a dummy column for the X-axis (Otherwise it will always be in separated columns) 
df_ct['Customer_t'] = "Customer Type"
df_ct

,Customer Type,Count,Customer_t
0,Member,163,Customer Type
1,Normal,140,Customer Type


In [13]:
fig = px.bar(df_ct,
             x="Customer_t",
             y="Count",
             color="Customer Type",
             barmode='stack',
             )
fig.update_layout(title='Customer Type in February')
fig.update_xaxes(title='')
fig

In [14]:
df["Product line"].unique()

array(['Health and beauty', 'Electronic accessories',
       'Home and lifestyle', 'Sports and travel', 'Food and beverages',
       'Fashion accessories'], dtype=object)

In [15]:
solar_colors = ['#FFDF00', '#FF4D00', '#007FFF', '#4F7942', '#FF8000', '#4B0082']
cmap_prod = dict(zip(df["Product line"].unique(),solar_colors))
cmap_prod

{'Health and beauty': '#FFDF00',
 'Electronic accessories': '#FF4D00',
 'Home and lifestyle': '#007FFF',
 'Sports and travel': '#4F7942',
 'Food and beverages': '#FF8000',
 'Fashion accessories': '#4B0082'}

In [16]:
solar_colors = ['#FFDF00', '#FF4D00', '#007FFF', '#4F7942', '#FF8000', '#4B0082']
cmap_gndr = dict(zip(df["Gender"].unique(),solar_colors[:2]))
cmap_gndr

{'Female': '#FFDF00', 'Male': '#FF4D00'}

In [17]:
#Sankey chart aka Parallel Categories
#If we do not aggregate, the result will be... a mess 
filtered = df[df["Date"].dt.month == 2]
fig = px.parallel_categories(filtered, 
                             dimensions=['Gender', 'Product line', 'City'],
                             color=filtered['Product line'].map(cmap_prod), 
                             #color_continuous_scale=px.colors.sequential.Inferno,
                             labels={'Gender':'Oh Gender', 'Product line':'What u buin´?', 'Total':'What total?'})
fig.show()

In [18]:
#Sankey chart aka Parallel Categories
#If we do not aggregate, the result will be... a mess 
filtered = df[df["Date"].dt.month == 2].round()
fig = px.parallel_categories(filtered, 
                             dimensions=['Product line', 'Gender', 'City',"Rating"],
                             color=filtered['Gender'].map(cmap_gndr), 
                             #color_continuous_scale=px.colors.sequential.Inferno,
                             labels={'Gender':'Oh Gender', 'Product line':'What u buin´?', 'Total':'What total?',"Rating":"Do u hate us?"})
fig.show()

In [19]:
#Sankey chart aka Parallel Categories
#If we do not aggregate, the result will be... a mess 
filtered = df[df["Date"].dt.month == 2].round()
fig = px.parallel_categories(filtered, 
                             dimensions=['Gender',"Rating"],
                             color=filtered['Rating'], 
                             color_continuous_scale=px.colors.sequential.Inferno,
                             labels={'Gender':'Oh Gender', 'Product line':'What u buin´?', 'Total':'What total?',"Rating":"Do u hate us?"})
fig.show()

In [20]:
fig = px.histogram(df[df["Date"].dt.month == 2], x="Rating", color="Gender", marginal="rug")
fig.show()

In [21]:
colors = ['#835AF1', '#7FA6EE', '#B8F7D4']
color_map = {
    'Female': '#835AF1',
    'Male': '#B8F7D4'
            }

fig = px.histogram(df, 
                   x="Rating", 
                   color = "Gender",
                   histnorm='probability density',
                   color_discrete_map=color_map,
                   marginal="rug", 
                   nbins=20
                   )
#ig.update_layout(bargap=0.05)
fig

In [22]:
filtered = df[df["Date"].dt.month == 2].round()
fig = px.sunburst(
    filtered,
    path=['Gender',"City",'Product line',],
    values='Total',
)
fig

In [23]:
df[df['Date'].dt.month_name() == "February"][df["City"] == "Yangon"]

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\480430127.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.21,634.38,2019-02-08,10:37,Ewallet,604.17,4.76,30.21,5.3
6,355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.84,6,20.65,433.69,2019-02-25,14:36,Ewallet,413.04,4.76,20.65,5.8
12,365-64-0515,A,Yangon,Normal,Female,Electronic accessories,46.95,5,11.74,246.49,2019-02-12,10:25,Ewallet,234.75,4.76,11.74,7.1
13,252-56-2699,A,Yangon,Normal,Male,Food and beverages,43.19,10,21.60,453.50,2019-02-07,16:48,Ewallet,431.90,4.76,21.60,8.2
23,636-48-8204,A,Yangon,Normal,Male,Electronic accessories,34.56,5,8.64,181.44,2019-02-17,11:15,Ewallet,172.80,4.76,8.64,9.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,324-92-3863,A,Yangon,Member,Male,Electronic accessories,20.89,2,2.09,43.87,2019-02-05,18:45,Cash,41.78,4.76,2.09,9.8
973,531-56-4728,A,Yangon,Normal,Male,Home and lifestyle,80.08,3,12.01,252.25,2019-02-11,15:29,Cash,240.24,4.76,12.01,5.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.59,33.43,2019-02-09,13:22,Cash,31.84,4.76,1.59,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.29,69.11,2019-02-22,15:33,Cash,65.82,4.76,3.29,4.1


In [24]:
dff = df[df['Date'].dt.month_name() == "February"][df["City"] == "Yangon"]
dffs = dff.groupby('Date').agg(Customers_Day=('Customer type', 'count'))
dffs.head()

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\3919046596.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Customers_Day
Date,
2019-02-01,3
2019-02-02,4
2019-02-03,5
2019-02-04,5
2019-02-05,5


In [25]:
dffs.reset_index().head()

,Date,Customers_Day
0,2019-02-01,3
1,2019-02-02,4
2,2019-02-03,5
3,2019-02-04,5
4,2019-02-05,5


In [26]:
#Customers per day in February
fig = px.bar(df[df["Date"].dt.month == 2][df["City"] == "Yangon"].groupby('Date').agg(Customers_per_Day=('Customer type', 'count')).reset_index(),
             x="Date",
             y="Customers_per_Day",
             #text = "Customer type",
             barmode='group',
             #text_auto=".2s"
             )
fig.show()

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\2306867145.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [27]:
df[df["Date"].dt.month_name() == "February"][df["City"] == "Yangon"].groupby('Date').agg(Sales_per_Day=('Total', 'sum')).reset_index().head()

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\1335995330.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Date,Sales_per_Day
0,2019-02-01,911.94
1,2019-02-02,1019.96
2,2019-02-03,2717.36
3,2019-02-04,1239.39
4,2019-02-05,568.43


In [28]:
dffs = df[df["Date"].dt.month_name() == "February"][df["City"] == "Yangon"].groupby('Date').agg(Sales=('Total', 'sum')).reset_index()
fig = px.bar(dffs,
             x="Date",
             y="Sales",
             barmode='group',
             #template = "flatly"
             )
fig.show()

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\1806506982.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [29]:
dffs = df[df["Date"].dt.month_name() == "February"][df["City"] == "Yangon"].groupby('Date').agg(Sales=('Total', 'sum'),
                                                                                                Gross_Income = ('gross income', 'sum'),
                                                                                                COGS = ('cogs', 'sum') ).reset_index()
dffs.head()

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\464304390.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Date,Sales,Gross_Income,COGS
0,2019-02-01,911.94,43.43,868.51
1,2019-02-02,1019.96,48.57,971.39
2,2019-02-03,2717.36,129.40,2587.96
3,2019-02-04,1239.39,59.02,1180.37
4,2019-02-05,568.43,27.06,541.36


In [30]:
fig1 = px.bar(dffs,
             x="Date",
             y=["Sales"],
             barmode='group',
             #template = "flatly"
             )
fig1

In [31]:
color_map = {
    'Gross_Income': 'green',
    'COGS': 'orange'
            }   

fig2 = px.bar(dffs,
             x="Date",
             y=["Gross_Income","COGS"],
             barmode='stack',
             color_discrete_map=color_map
             )
fig2

In [32]:
#https://stackoverflow.com/questions/70563166/stacked-barplot-in-plotly

#Combining both
fig3 = go.Figure(
            data=[
                go.Bar(
                    name="Sales",
                    x=dffs["Date"],
                    y=dffs["Sales"],
                    offsetgroup=0,
                ),
                go.Bar(
                    name="COGS",
                    x=dffs["Date"],
                    y=dffs["COGS"],
                    offsetgroup=1,
                ),
                go.Bar(
                    name="Income",
                    x=dffs["Date"],
                    y=dffs["Gross_Income"],
                    offsetgroup=1,
                    base=dffs["COGS"], 
                )
                ],
                layout=go.Layout(
                    title="Sales, COGS & Gross Income",
                    yaxis_title="Value"
                )
                )
fig3.show()

#Unfortunately this breaks the dash app when trying to render it  ))))))))))))))):

In [33]:
dffs = df[df["Date"].dt.month_name() == "February"][df["City"] == "Yangon"].groupby(['Date',"Product line"]).agg(Sales=('Total', 'sum'),
                                                                                                #Product_type = ('Product line', 'count')
                                                                                                ).reset_index()
dffs.head()

C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\1674913371.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Date,Product line,Sales
0,2019-02-01,Food and beverages,74.71
1,2019-02-01,Health and beauty,326.26
2,2019-02-01,Sports and travel,510.97
3,2019-02-02,Fashion accessories,452.86
4,2019-02-02,Food and beverages,115.08


In [34]:
fig = px.bar(dffs,
             x="Date",
             y="Sales",
             color= "Product line",
             barmode='stack',
             )
fig

In [35]:
string = "Customer"
string[0:4]

'Cust'

In [36]:
#Total gross income for February in Mandalay...
dff = df[df["Date"].dt.month_name() == "February"][df["City"] == "Mandalay"]
dff["gross income"].sum().round()


C:\Users\XandorTheWise\AppData\Local\Temp\ipykernel_18564\2236753228.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



1639.0

In [37]:
#Month = February, City = Mandalay
dffs = dff.groupby(['Gender']).agg(Genders=('Gender', 'count')).reset_index()
dffs   

,Gender,Genders
0,Female,55
1,Male,54


In [38]:
fig = px.pie(dffs,
             values= "Genders",
             names="Gender",
             title='Genders in Mandalay, February',
             hole=.3)
fig.update_traces(textposition='inside',
                  textinfo='percent+value')

fig 

In [39]:
#Month = February, City = Mandalay
dffs = dff.groupby(['Payment']).agg(Type=('Payment', 'count')).reset_index()
dffs   

,Payment,Type
0,Cash,44
1,Credit card,30
2,Ewallet,35


In [40]:
fig = px.pie(dffs,
             values= "Type",
             names="Payment",
             title='Type of Payment, Mandalay - February',
             hole=.3)
fig.update_traces(textposition='inside',
                  textinfo='percent+value')

fig 

In [41]:
dffs = dff.groupby(['Customer type']).agg(Type_c=('Customer type', 'count')).reset_index()
dffs["Type"] = "Type"
dffs

,Customer type,Type_c,Type
0,Member,56,Type
1,Normal,53,Type


In [47]:

# Step 2: Calculate the total count
total_count = dffs['Type_c'].sum()

# Step 3: Compute the relative percentage for each group
dffs['Percentage'] = (dffs['Type_c'] / total_count) * 100
dffs['Percentage'] = dffs['Percentage'].round(2)
dffs

,Customer type,Type_c,Type,Percentage
0,Member,56,Type,51.38
1,Normal,53,Type,48.62


In [56]:
fig = px.bar(dffs,
             y="Type",
             x="Type_c",
             color="Customer type",
             orientation= "h",
             barmode='stack',
             )
fig.update_layout(title='Customer Type in February')
fig.update_xaxes(title='')
fig.update_yaxes(title='')
fig